In [1]:
import tensorflow as tf
tf.__version__


'2.12.0'

+ 해당 실습에 사용되는 데이터 셋은 트레이닝 데이터 600개, 테스트 데이터 100개로 구성

- Batch size
    * mini batch 크기 (트레이닝 데이터를 쪼개는 단위)
    * mini batch 하나 당 파라미터 한번 업데이트
    * 트레이닝 데이터를 통째로 신경망에 넣으면 비효율적 리소스 사용으로 학습시간 증가
    * ==> 트레이닝 셋 작게 나눈다
- Epoch
    * 전체 트레이닝 셋이 신경망을 통과한 횟수
    * ex) 1 epoch = Forward, Backward propagation 이 신경망을 통해 한번 진행됨
- Iteration
    * 1 epoch 를 마치는데 필요한 미니 배치 개수 (1 epoch를 마치는데 필요한 파라미터 업데이트 횟수)
    * ex) 트레이닝 데이터 = 600개, Batch size = 10 ==> Iteration = 60 (1 epoch 시 paramater 60번 업데이트)

- Learning rate (0~1 사이 값)
    * loss function을 토대로 계산한 gradient 값에 learning rate 곱해서 원래 가중치에서 뺀다 

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2

tr_file = "./list_tr1"
ts_file = "./list_ts1"
n_epochs = int(input("Enter number of epochs : "))
b_size = int(input("Enter batch size : "))      
n_tag = 10
l_rate = float(input("Enter learning rate : "))
SX = SY = 64    # 데이터 이미지 크기 : 64 x 64

def Getdata(list_file):
    fp = open(list_file, 'r')               
    lines = fp.read().splitlines()      # line 단위로 파일 읽는다        
    inp = []
    tag = []
    
    for line in lines:
        img_file, id = line.split(' ')  # 공백을 기준으로 split (img_file:파일 경로, id:target (class 정보))
        #print(img_file, id)
        img = cv2.imread(img_file)       
        img = cv2.resize(img, dsize=(SY, SX)) # 64x64로 resize
        inp.append(img/255)             # image data (0~255) 값을 0~1 값으로 Normalize
        tag.append(np.uint8(id))        # String 2 integer
        
    fp.close()
    return inp, tag     # list 반환

# Training dataset 불러오기 (tr)
[tr_inp, tr_tag] = Getdata(tr_file)
tr_inp = np.array(tr_inp)
tr_tag = np.array(tr_tag)

# Test dataset 불러오기 (ts)
[ts_inp, ts_tag] = Getdata(ts_file)
ts_inp = np.array(ts_inp)
ts_tag = np.array(ts_tag)

#  [Layer 구성]  #
model = models.Sequential()                                                         # layer 모델 초기화 (sequential : 연속적 레이어)
model.add(layers.Conv2D(4,(8,8), activation='relu', input_shape=(64,64,3)))         # (11x11) 5개, 입력 데이터:64x64x3 (image라 channel 3개)
model.add(layers.MaxPooling2D((2,2)))                                                   # 4x4 단위로 maxpooling
model.add(layers.Conv2D(8,(4,4), activation='relu'))                                    # 2D conv (7x7) 7개 
model.add(layers.MaxPooling2D((2,2)))  
model.add(layers.Flatten())                                                             # 1차원으로 재배치
model.add(layers.Dense(256, activation='relu'))                                         # MLP (Multi-layer perceptron), 노드 개수:128  
model.add(layers.Dense(n_tag, activation='softmax'))                            # MLP, 최종 노드 개수는 class 개수랑 동일해야 = n_tag -> linear이면 Regression

model.summary()  # 화면에 layer 표시

#  [학습, 테스트]  #
adam = optimizers.Adam(learning_rate = l_rate)  
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])   # adam 알고리즘, loss 계산은 advanced crossentropy, 적중률
model.fit(tr_inp, tr_tag, batch_size=b_size, epochs=n_epochs, verbose=1, shuffle=1)     # 학습 시작 (training dataset 입력, batch size, epoch 입력)
test_loss, test_acc = model.evaluate(ts_inp, ts_tag, verbose=0)  # 모델 테스트 : loss, 정확도(적중률) 측정 -> loss는 낮을수록 좋고, acc는 높을수록 좋다
print("Results for test data=")
print(test_acc)  # 테스트 결과(accuracy) 출력 

model.save("model_7.h5")

#  [Confusion matrix 제공]  # 
#  테스트 결과 출력 : class별로 어떤걸로 판단했는지 횟수 제공  # 
pred = model.predict(ts_inp)
y_pred = np.argmax(pred, axis=1)
conf_mat = confusion_matrix(ts_tag, y_pred)
print(conf_mat)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 57, 57, 4)         772       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 28, 28, 4)        0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 25, 25, 8)         520       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 12, 12, 8)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 1152)              0         
                                                                 
 dense_8 (Dense)             (None, 256)              

In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2

SY = SX = 64

def Getdata(list_file):
    fp = open(list_file, 'r')               
    lines = fp.read().splitlines()      # line 단위로 파일 읽는다        
    inp = []
    tag = []
    
    for line in lines:
        img_file, id = line.split(' ')  # 공백을 기준으로 split (img_file:파일 경로, id:target (class 정보))
        img = cv2.imread(img_file)   
        img = cv2.resize(img, dsize=(SY, SX)) # 64x64로 resize
        inp.append(img/255)             # image data (0~255) 값을 0~1 값으로 Normalize
        tag.append(np.uint8(id))        # String 2 integer
        
    fp.close()
    return inp, tag     # list 반환

model = load_model("model_7.h5")

model.summary()

ts_file = "./list_tr3"
[ts_inp2, ts_tag2] = Getdata(ts_file)
ts_inp2 = np.array(ts_inp2)
ts_tag2 = np.array(ts_tag2)

test_loss, test_acc = model.evaluate(ts_inp2, ts_tag2, verbose=0)  # 모델 테스트 : loss, 정확도(적중률) 측정 -> loss는 낮을수록 좋고, acc는 높을수록 좋다
print("Results for test data=")
print(test_acc)  # 테스트 결과(accuracy) 출력 

pred = model.predict(ts_inp2)
y_pred = np.argmax(pred, axis=1)
conf_mat = confusion_matrix(ts_tag2, y_pred)
print(conf_mat)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 54, 54, 5)         1820      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 5)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 7)           1722      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 1, 7)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 7)                 0         
                                                                 
 dense_2 (Dense)             (None, 128)              

### Epoch = 30, Batch size = 10, Learning rate = 0.01

 - Results for test data=
 - 0.8833333253860474
 - 10/10 [==============================] - 0s 9ms/step
 - [[38 12  0  0  0  0]
 -  [ 0 50  0  0  0  0]
 -  [ 0  0 36 14  0  0]
 -  [ 0  0  2 48  0  0]
 -  [ 0  0  0  0 43  7]
 -  [ 0  0  0  0  0 50]]
 

---

### Epoch = 50, Batch size = 15, Learning rate = 0.03
- Results for test data=
- 0.9900000095367432
- 10/10 [==============================] - 0s 8ms/step
- [[49  0  0  0  0  1]
-  [ 0 50  0  0  0  0]
-  [ 0  0 50  0  0  0]
-  [ 0  0  2 48  0  0]
-  [ 0  0  0  0 50  0]
-  [ 0  0  0  0  0 50]]